<a href="https://colab.research.google.com/github/NNFLgroup31/Resnet50-Hyperparametertuning/blob/master/FP16_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Packages And Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import torchvision
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from torch.optim.lr_scheduler import LambdaLR

##Input Parameter

In [0]:
epochs=120
bs=128
lr=0.1
model_save_name = 'Fp16_imagenet_Resnet-50.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"


## Model and Optimizer Intialization

In [0]:
def get_model():
 model=torchvision.models.resnet50(pretrained=False) ##Resnet-50##
 #model=torchvision.models.mobilenet_v2(pretrained=False) ##Mobilenet_v2##
 return model,optim.SGD(model.parameters(),lr=lr,momentum=0.9,nesterov=True)
model,opt=get_model()

model.fc=nn.Linear(2048,10,bias=True) ##Resnet50##
#model._modules['classifier']._modules['1']=nn.Linear(1280,10,bias=True) ##Mobilenet_v2##
for layer in model.modules():
  if isinstance(layer,nn.Conv2d):
    torch.nn.init.xavier_uniform(layer.weight)
  if isinstance(layer,nn.Linear):
    torch.nn.init.xavier_uniform(layer.weight)
  if isinstance(layer,nn.BatchNorm2d):
    layer.weight.data.fill_(1)
    layer.bias.data.zero_()

##FP16

In [0]:
model.half()
for layer in model.modules():
  if isinstance(layer,nn.BatchNorm2d):
    layer.float()
model.cuda()

## Checkpoint

In [0]:
#checkpoint={}
history=[]
#checkpoint=torch.load(path)
epoch_last=1
#model.load_state_dict(checkpoint['model_state_dict'])
#opt.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch_last=checkpoint['epoch']
#history=checkpoint['history']

## Scheduler 

In [0]:
def x(epoch):
  if (epoch < 30):
     return 1
  elif (epoch >= 30) and (epoch<60):
     return 0.1
  elif  (epoch >= 60) and (epoch<90):
     return 0.01
  elif epoch >= 90:
     return 0.001

lambda1 = lambda epoch: x(epoch)
scheduler = LambdaLR(opt, [lambda1])
#scheduler.load_state_dict(checkpoint['scheduler'])

## PCA Noise EigenValues,EigenVectors

In [0]:
__imagenet_pca = {
    'eigval': torch.Tensor([0.2175, 0.0188, 0.0045]),
    'eigvec': torch.Tensor([
        [-0.5675,  0.7192,  0.4009],
        [-0.5808, -0.0045, -0.8140],
        [-0.5836, -0.6948,  0.4203],
    ])
}

# PCA_Gaussian_noise
class Lighting(object):
    

    def __init__(self, alphastd, eigval, eigvec):
        self.alphastd = alphastd
        self.eigval = eigval
        self.eigvec = eigvec

    def __call__(self, img):
        if self.alphastd == 0:
            return img

        alpha = img.new().resize_(3).normal_(0, self.alphastd)
        rgb = self.eigvec.type_as(img).clone()\
            .mul(alpha.view(1, 3).expand(3, 3))\
            .mul(self.eigval.view(1, 3).expand(3, 3))\
            .sum(1).squeeze()
        return img.add(rgb.view(3, 1, 1).expand_as(img))

## Accuracy(Topk) & Loss Fuction

In [0]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k)
    return res

loss_func=F.cross_entropy
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    #print(loss)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

## Mean & Standard Deviation of Dataset

In [0]:
def get_mean_and_std(train_dir):
    '''Compute the mean and std value of dataset.'''
    dataset = datasets.ImageFolder(train_dir,transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),torchvision.transforms.ColorJitter(brightness=(0.6,1.4),saturation=(0.6,1.4),hue=(-0.5,0.5)),transforms.ToTensor(), Lighting(0.1, __imagenet_pca['eigval'], __imagenet_pca['eigvec'])]))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:,i,:,:].mean()
            std[i] += inputs[:,i,:,:].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

## Dataset

In [0]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz ##Imagenette2##
!tar -xvzf  'imagenette2.tgz'

#!wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2.tgz ##Imagewoof2##
#!tar -xvzf  'imagewoof2.tgz' 

## Preprocessing & DataLoader

In [0]:
train_dir=os.path.join('/content/gdrive/My Drive/imagenette2/','train') #Imagenette2##
val_dir=os.path.join('/content/gdrive/My Drive/imagenette2/','val')

#train_dir=os.path.join('/content/imagewoof2','train') ##Imagewoof2##
#val_dir=os.path.join('/content/imagewoof2','val')       

mean_dataset,std_dataset=get_mean_and_std(train_dir)      
normalize = transforms.Normalize(mean=mean_dataset,std=std_dataset)
train_dataset = datasets.ImageFolder(train_dir,transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),torchvision.transforms.ColorJitter(brightness=(0.6,1.4),saturation=(0.6,1.4),hue=(-0.5,0.5)),transforms.ToTensor(), Lighting(0.1, __imagenet_pca['eigval'], __imagenet_pca['eigvec']),normalize]))
train_loader=DataLoader(train_dataset,batch_size=bs,shuffle=True,pin_memory=True)
val_loader=DataLoader(datasets.ImageFolder(val_dir,transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),normalize])),batch_size=bs,shuffle=False,pin_memory=True)


## Training & Validation

In [0]:
for epoch in range(epoch_last,epochs+1):
        model.train()
        t_loss=0
        t_num=0
        for xb, yb in train_loader:
            xb=xb.to('cuda').half()
            yb=yb.to('cuda')
            a,b=loss_batch(model, loss_func, xb, yb, opt)
            t_loss += a*b
            t_num += b
        train_loss=t_loss/t_num


        model.eval()
        losses=0
        nums=0
        correct1=0
        correct2=0
        correct3=0
        with torch.no_grad():
            for xb, yb in val_loader:
              xb=xb.to('cuda').half()
              yb=yb.to('cuda')
              l,c=loss_batch(model, loss_func, xb, yb)
              losses +=l*c
              nums += c
              correct_class1,correct_class2,correct_class3 = accuracy(model(xb),yb,topk=(1,3,5))
              correct1 += int(correct_class1)
              correct2 += int(correct_class2)
              correct3 += int(correct_class3)
        val_loss = losses / nums
        acc1=correct1/nums
        acc2=correct2/nums
        acc3=correct3/nums
        history.append([epoch,train_loss,val_loss,acc1,acc2,acc3])
        scheduler.step()
        print(epoch,train_loss,val_loss,acc1,acc2,acc3)
        torch.save({'epoch':epoch,'model_state_dict': model.state_dict(),'optimizer_state_dict': opt.state_dict(),'history':history,'scheduler':scheduler.state_dict()}, path)